## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [ ]:
load_dotenv(override=True)

In [ ]:
print("SENDGRID_API_KEY present?", bool(os.environ.get('SENDGRID_API_KEY')))

In [ ]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("michaelmmckenzie33@gmail.com")  # Change to your verified sender
    to_email = To("michaelmmckenzie33@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("michaelmmckenzie33@gmail.com")  # Change to your verified sender
    to_email = To("michaelmmckenzie33@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [ ]:
# This code is primarily for testing when an expected email is not received.
# In many cases, emails can fail silently if you do not inspect SendGrid’s API
# response. Issues such as an invalid API key, unverified sender address, or
# rate limiting may cause the request to be rejected without visible errors
# unless the response is explicitly captured and reviewed.
#
# The send_email function below has been updated to return full response details
# from SendGrid — including the status code, response body, and headers — so you
# can see exactly what happened during the send attempt.
#
# Note: In some scenarios, the Sales Manager agent may call send_email multiple
# times (for example, once for each generated draft) instead of only once after
# selecting the best email. This behavior will result in multiple emails being
# sent unless restricted in the instructions or in code.

# 1) Redefine the tool (VERIFIED sender)
from agents import Agent, Runner, trace, function_tool
import os, sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

@function_tool
def send_email(body: str):
    """Send out an email with the given body to all sales prospects."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ["SENDGRID_API_KEY"])
    from_email = Email("michaelmmckenzie33@gmail.com")  # verified sender
    to_email   = To("michaelmmckenzie33@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    resp = sg.client.mail.send.post(request_body=mail)
    # Return details instead of only printing
    return {
        "status": "success" if resp.status_code == 202 else "unexpected_status",
        "code": resp.status_code,
        "body": resp.body.decode("utf-8") if hasattr(resp.body, "decode") else str(resp.body),
        "headers": dict(resp.headers),
    }

# 2) Rebuild tools AFTER redefining the function
description = "Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)
tools = [tool1, tool2, tool3, send_email]

# 3) Recreate the manager with those tools
instructions = """
You are a Sales Manager at ComplAI.
Steps:
1) Call ALL THREE sales_agent tools to draft three emails.
2) Pick the best one.
3) You MUST call the send_email tool exactly once with the chosen body.
Return 'DONE' only after send_email returns code 202.
"""
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

# 4) Run and capture the tool result
message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

# Optional: inspect the result structure if available in your SDK
print("Final output:", getattr(result, "final_output", None))
print("All tools on agent:", [t.name for t in sales_manager.tools])



In [ ]:
# This is a minimal SendGrid test, useful for verifying that email sending works
# when you are not receiving messages as expected.
# 
# In this test, the code connects to SendGrid, authenticates with the provided
# API key, and submits an email request. A status code of 202 (“Accepted”)
# means that SendGrid successfully received the request and queued the email
# for delivery. It does not confirm that the email has been delivered, only that
# SendGrid will attempt to send it. 
# 
# The output also includes HTTP response headers returned by SendGrid’s API,
# which provide metadata such as the unique message ID, security policies,
# and connection details.

from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

sg = SendGridAPIClient(os.environ["SENDGRID_API_KEY"])
msg = Mail(
    from_email="michaelmmckenzie33@gmail.com",
    to_emails="michaelmmckenzie33@gmail.com",
    subject="Hello from SendGrid",
    plain_text_content="Test"
)
resp = sg.send(msg)
print(resp.status_code, resp.body, resp.headers)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("michaelmmckenzie33@gmail.com")  # Change to your verified sender
    to_email = To("michaelmmckenzie33@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>


##### **Agentic design patterns in your code**


*   **Ensemble generation (fan-out/fan-in):** three specialized writers (sales\_agent1/2/3) generate drafts in parallel (asyncio.gather), then you aggregate results.
    
*   **Ranker/selector:** a separate sales\_picker agent chooses the single best draft from the ensemble.
    
*   **Agents-as-tools:** sales\_agentX.as\_tool(...) exposes agents as callable tools so another agent can invoke them.
    
*   **Tool use / function calling:** @function\_tool wraps send\_email (and later send\_html\_email) so agents can call real actions.
    
*   **Handoff pattern:** the Sales Manager _hands off_ the winning draft to the Email Manager agent, which then uses subject\_writer, html\_converter, and send\_html\_email.
    
*   **Tracing/observability:** with trace(...) brackets major phases to produce a timeline of steps/events.
    
*   **Streaming UX:** Runner.run\_streamed(...) surfaces incremental deltas for a responsive UI.
    

##### **The 1 line that shifts “workflow” → “agent” (Anthropic sense)**

Anthropic treats an **agent** as something that can decide _which tools to use, in what order, to achieve a goal_. Your code becomes agentic at the moment you give a model tools and goals, not just a fixed pipeline.

**That line is:**

```python
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")
```


Up to that point you had an orchestrated **workflow** (you decided the sequence with asyncio.gather, then ranking). With this line, the Sales Manager can plan and choose tools autonomously—i.e., it’s now an **agent**.

##### **Ideas to add more tools & agents**


*  **Mail-merge tool:** loads a CSV of prospects and sends the chosen email to each (respects rate limits, dedupes, tracks per-recipient message IDs).
    
*   **Personalizer agent:** given a prospect record (name, company, ICP fit, last news), returns line-level personalization to prepend to the chosen draft.
    
*   **Safety/review tool:** checks for compliance/PII/claims; blocks send if risky.
    
*   **A/B testing tool:** splits list into cohorts, tracks SendGrid metrics (opens/clicks) via Event Webhook, and recommends the winner.
    
*   **CRM tool:** logs the sent email + message ID to your CRM (HubSpot/Salesforce) and updates contact status.
    

Sketch for mail-merge tool:

In [ ]:
# drop-in mail-merge send tool you can use with your existing agent setup. It reads a CSV of recipients, formats a subject/body 
# with per-row fields (e.g., {first_name}, {company}), sends via SendGrid with basic retry/rate-limit handling, and returns a 
# detailed summary (including X-Message-Id per recipient).

from agents import function_tool
from typing import Dict, List, Any, Optional
import csv, os, time, json, math
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

class _SafeDict(dict):
    # Avoid KeyError during .format_map(); missing keys render as {key}
    def __missing__(self, key): 
        return "{" + key + "}"

def _format_tmpl(tmpl: str, row: Dict[str, Any]) -> str:
    return tmpl.format_map(_SafeDict(**row))

def _sleep_for_rate(rate_per_sec: float, last_sent_ts: List[float]):
    if rate_per_sec <= 0:
        return
    min_interval = 1.0 / rate_per_sec
    now = time.time()
    elapsed = now - last_sent_ts[0]
    if elapsed < min_interval:
        time.sleep(min_interval - elapsed)
    last_sent_ts[0] = time.time()

def _send_one(
    sg: sendgrid.SendGridAPIClient,
    from_email: str,
    to_email: str,
    subject: str,
    body_text: Optional[str] = None,
    body_html: Optional[str] = None,
    headers: Optional[Dict[str, str]] = None,
) -> Dict[str, Any]:
    # prefer html if provided
    if body_html:
        content = Content("text/html", body_html)
    else:
        content = Content("text/plain", body_text or "")
    mail = Mail(Email(from_email), To(to_email), subject, content).get()

    if headers:
        # add custom headers like thread tokens, campaign ids, etc.
        mail.setdefault("headers", {}).update(headers)

    # Send with light retries for 429/5xx
    attempts, max_attempts, backoff = 0, 4, 1.0
    while True:
        attempts += 1
        resp = sg.client.mail.send.post(request_body=mail)
        code = getattr(resp, "status_code", None)
        if code and 200 <= code < 300:
            # Accepted (202) or 2xx
            x_msg_id = None
            try:
                x_msg_id = resp.headers.get("X-Message-Id") or resp.headers.get("x-message-id")
            except Exception:
                pass
            return {
                "status": "success",
                "code": code,
                "message_id": x_msg_id,
                "headers": dict(getattr(resp, "headers", {}) or {}),
                "body": (resp.body.decode("utf-8") if hasattr(resp.body, "decode") else str(resp.body)),
            }

        # Retry for transient issues
        if code in (429, 500, 502, 503, 504) and attempts < max_attempts:
            time.sleep(backoff)
            backoff *= 2
            continue

        # Permanent failure or exhausted retries
        return {
            "status": "error",
            "code": code,
            "headers": dict(getattr(resp, "headers", {}) or {}),
            "body": (resp.body.decode("utf-8") if hasattr(resp.body, "decode") else str(resp.body)),
        }

@function_tool
def mail_merge_send(
    recipients_csv_path: str,
    subject_template: str,
    body_template: Optional[str] = None,
    html_template: Optional[str] = None,
    from_email: Optional[str] = None,
    to_field: str = "email",
    preview_limit: int = 0,
    rate_limit_per_sec: float = 5.0,
    custom_header_key: Optional[str] = None,
    custom_header_template: Optional[str] = None,
    dry_run: bool = False
) -> Dict[str, Any]:
    """
    Send a personalized email to each row in a CSV using SendGrid.

    Parameters
    ----------
    recipients_csv_path : str
        Path to CSV with at least a column named 'email' (or set to_field).
        You can include arbitrary columns like first_name, company, etc.
    subject_template : str
        Subject template; e.g. "Hi {first_name}, quick question about {company}"
    body_template : str, optional
        Plain-text body template. Use either body_template or html_template (or both).
    html_template : str, optional
        HTML body template (preferred if present).
    from_email : str, optional
        Verified sender address. If None, uses SENDGRID_FROM_EMAIL env.
    to_field : str
        CSV column to use for recipient address (default 'email').
    preview_limit : int
        If >0, only process this many rows (useful for tests).
    rate_limit_per_sec : float
        Cap send rate to avoid API throttling (approximate).
    custom_header_key : str, optional
        If set, add a custom header per recipient (e.g., "X-Campaign-Id").
    custom_header_template : str, optional
        Template for the header value per row (e.g., "{campaign_id}:{uuid}").
    dry_run : bool
        If True, do not send—just return what would be sent.

    Returns
    -------
    Dict
        {
          "summary": {"attempted": N, "sent": S, "errors": E, "dry_run": bool},
          "items": [
             {
               "to": "...", "subject": "...",
               "status": "success"|"error"|"skipped",
               "code": 202,
               "message_id": "...",
               "error": "...",
               "preview": true|false
             }, ...
          ]
        }
    """
    # Resolve sender
    from_email = from_email or os.environ.get("SENDGRID_FROM_EMAIL")
    if not from_email:
        return {"summary": {"attempted": 0, "sent": 0, "errors": 0, "dry_run": dry_run},
                "items": [],
                "error": "Missing from_email (pass param or set SENDGRID_FROM_EMAIL)"}

    # Read CSV
    rows: List[Dict[str, Any]] = []
    with open(recipients_csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append({k: (v or "").strip() for k, v in r.items()})

    if preview_limit and preview_limit > 0:
        rows = rows[:preview_limit]

    # Init SendGrid client (even in dry_run we still format; we just don't send)
    api_key = os.environ.get("SENDGRID_API_KEY")
    if not api_key and not dry_run:
        return {"summary": {"attempted": 0, "sent": 0, "errors": 0, "dry_run": dry_run},
                "items": [],
                "error": "Missing SENDGRID_API_KEY env var"}

    sg = sendgrid.SendGridAPIClient(api_key=api_key) if not dry_run else None

    items: List[Dict[str, Any]] = []
    sent = errors = 0
    last_sent_ts = [0.0]

    for idx, row in enumerate(rows, start=1):
        to_addr = row.get(to_field, "").strip()
        if not to_addr:
            items.append({
                "row": idx, "to": None, "status": "skipped",
                "error": f"Missing '{to_field}' column value"
            })
            errors += 1
            continue

        subject = _format_tmpl(subject_template, row)
        text_body = _format_tmpl(body_template, row) if body_template else None
        html_body = _format_tmpl(html_template, row) if html_template else None

        hdrs = None
        if custom_header_key and custom_header_template:
            hdrs = {custom_header_key: _format_tmpl(custom_header_template, row)}

        if dry_run:
            items.append({
                "row": idx, "to": to_addr, "subject": subject,
                "status": "preview", "preview": True,
                "headers": hdrs,
                "body_text": text_body, "body_html": html_body
            })
            continue

        _sleep_for_rate(rate_limit_per_sec, last_sent_ts)

        result = _send_one(
            sg=sg,
            from_email=from_email,
            to_email=to_addr,
            subject=subject,
            body_text=text_body,
            body_html=html_body,
            headers=hdrs
        )

        item = {"row": idx, "to": to_addr, "subject": subject, **result}
        items.append(item)
        if result["status"] == "success":
            sent += 1
        else:
            errors += 1

    summary = {
        "attempted": len(rows),
        "sent": sent,
        "errors": errors,
        "dry_run": dry_run,
    }
    return {"summary": summary, "items": items}

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.